# Build Metabolic Model (Advanced)

To build a metabolic model we need the following inputs:
 - template (genome scale)
 - template (core)
 - atp metabolism medias - ModelSEEDpy provides an array of atp metabolism medias
 - genome
 - genome growth medium

In [2]:
import modelseedpy
from modelseedpy.core.mstemplate import MSTemplateBuilder

### Load Templates

In [5]:
import json
with open('./Core-V5.2.json') as fh:
    template_core = MSTemplateBuilder.from_dict(json.load(fh)).build()
with open('./GramNegModelTemplateV6.json') as fh:
    template_gramneg = MSTemplateBuilder.from_dict(json.load(fh)).build()

In [6]:
template_core

ID,CoreTemplateBacteria
Memory address,0x07fd278173730
Number of metabolites,197
Number of species,232
Number of reactions,252
Number of biomasses,0
Number of roles,706
Number of complexes,468


In [7]:
template_gramneg

ID,GramNegative.modeltemplate
Memory address,0x07fd10d9736d0
Number of metabolites,6573
Number of species,7116
Number of reactions,8584
Number of biomasses,1
Number of roles,20548
Number of complexes,3296


### Load Genome and Annotate with RAST

In [8]:
genome = modelseedpy.MSGenome.from_fasta('/opt/python/ModelSEEDpy/examples/Model Reconstruction/GCF_000005845.2_ASM584v2_protein.faa')

In [9]:
rast = modelseedpy.RastClient()
out = rast.annotate_genome(genome)

In [10]:
for feature in genome.features:
    print(feature.ontology_terms)

{'RAST': ['Thr operon leader peptide']}
{'RAST': ['Aspartokinase (EC 2.7.2.4)', 'Homoserine dehydrogenase (EC 1.1.1.3)']}
{'RAST': ['Homoserine kinase (EC 2.7.1.39)']}
{'RAST': ['Threonine synthase (EC 4.2.3.1)']}
{'RAST': ['Uncharacterized protein YaaX']}
{'RAST': ['UPF0246 protein YaaA']}
{'RAST': ['Putative alanine/glycine transport protein']}
{'RAST': ['Transaldolase (EC 2.2.1.2)']}
{'RAST': ['Molybdopterin adenylyltransferase (EC 2.7.7.75)']}
{'RAST': ['Succinate-acetate/proton symporter SatP']}
{'RAST': ['UPF0174 protein YaaW']}
{'RAST': ['UPF0412 protein YaaI']}
{'RAST': ['Chaperone protein DnaK']}
{'RAST': ['Chaperone protein DnaJ']}
{'RAST': ['Transposase InsL for insertion sequence element IS186']}
{}
{'RAST': ['Na+/H+ antiporter NhaA type']}
{'RAST': ['Transcriptional activator NhaR']}
{'RAST': ['IS1 protein InsB']}
{'RAST': ['IS1 protein InsA']}
{'RAST': ['SSU ribosomal protein S20p']}
{'RAST': ['Uncharacterized protein YaaY']}
{'RAST': ['FMN adenylyltransferase (EC 2.7.7.2

### 1 - Build Base Model

In [13]:
builder = modelseedpy.MSBuilder(genome, template_gramneg, 'test')

In [14]:
model_base = builder.build_base_model('test', annotate_with_rast=False)

In [15]:
builder.add_atpm(model_base)
model_base

Name,test
Memory address,7fd10924b100
Number of metabolites,1502
Number of reactions,1830
Number of genes,1338
Number of groups,1077
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"Cytosol, Extracellular"


### 2 - ATP Gapfill / Analysis

In [16]:
from modelseedpy.core.msatpcorrection import load_default_medias

In [17]:
default_medias = load_default_medias()
print(f'loaded {len(default_medias)} medias')

loaded 54 medias


In [18]:
for media, min_obj in default_medias:
    print(media.id, media.get_media_constraints())
    break

Glc.O2 {'cpd00027_e0': (-1, 1000), 'cpd00007_e0': (-1000, 1000), 'cpd00001_e0': (-1000, 1000), 'cpd00067_e0': (-1000, 1000)}


In [19]:
from modelseedpy import MSATPCorrection
atp_correction = MSATPCorrection(model_base, template_core, default_medias,
                                 compartment='c0', atp_hydrolysis_id='ATPM_c0', 
                                 load_default_medias=False)

In [20]:
media_eval = atp_correction.evaluate_growth_media()
atp_correction.determine_growth_media()
atp_correction.apply_growth_media_gapfilling()
atp_correction.expand_model_to_genome_scale()
tests = atp_correction.build_tests()

No gapfilling solution found before filtering for Etho activating ATPM_c0
No gapfilling solution found before filtering for mal-L activating ATPM_c0
No gapfilling solution found before filtering for Pyr.SO4 activating ATPM_c0
No gapfilling solution found before filtering for H2.SO4 activating ATPM_c0
No gapfilling solution found before filtering for empty activating ATPM_c0
No gapfilling solution found before filtering for Light activating ATPM_c0
No gapfilling solution found before filtering for ANME activating ATPM_c0
No gapfilling solution found before filtering for Methane activating ATPM_c0


In [21]:
for t in tests:
    print(t['media'].id, t['threshold'], atp_correction.media_gapfill_stats[t['media']])

empty 1e-05 None
Glc.O2 31.800000000002317 {'reversed': {}, 'new': {}}
Ac.O2 8.999999999999318 {'reversed': {}, 'new': {}}
Etho.O2 14.7 {'reversed': {}, 'new': {}}
Pyr.O2 12.45 {'reversed': {}, 'new': {}}
Glyc.O2 18.15000000000082 {'reversed': {}, 'new': {}}
Fum.O2 16.2 {'reversed': {}, 'new': {}}
Akg.O2 21.900000000000272 {'reversed': {}, 'new': {}}
LLac.O2 14.700000000000681 {'reversed': {}, 'new': {}}
Dlac.O2 14.699999999999317 {'reversed': {}, 'new': {}}
For.O2 2.2500000000002727 {'reversed': {}, 'new': {}}
Glc 6.0 {'reversed': {}, 'new': {}}
Pyr 1.2 {'reversed': {}, 'new': {}}
Glyc 1.2 {'reversed': {}, 'new': {}}
Fum 2.1 {'reversed': {}, 'new': {}}
Llac 0.6 {'reversed': {}, 'new': {}}
Dlac 0.6 {'reversed': {}, 'new': {}}
For.NO2 0.09999999999990905 {'reversed': {}, 'new': {}}
Pyr.NO3 4.95 {'reversed': {}, 'new': {}}
Ac.NO2 0.39999999999990904 {'reversed': {}, 'new': {}}
Ac.NO3 3.0 {'reversed': {}, 'new': {}}
Glc.DMSO 4.800000000002319 {'reversed': {}, 'new': {}}
Glc.TMAO 4.8 {'rev

### 3 - Genome Scale Gapfilling

In [24]:
from modelseedpy import MSMedia
media_glucose = MSMedia.from_dict({
    'cpd00149': (-100, 100),
    'cpd00099': (-100, 100),
    'cpd00067': (-100, 100),
    'cpd00063': (-100, 100),
 'cpd00058': (-100, 100),
 'cpd00048': (-100, 100),
 'cpd00034': (-100, 100),
 'cpd00030': (-100, 100),
 'cpd00013': (-100, 100),
 'cpd00009': (-100, 100),
 'cpd00001': (-100, 100),
 'cpd00007': (-10, 100),
 'cpd00205': (-100, 100),
 'cpd00254': (-100, 100),
 'cpd00971': (-100, 100),
 'cpd10515': (-100, 100),
 'cpd10516': (-100, 100),
 'cpd11574': (-100, 100),
 'cpd00244': (-100, 100),
 'cpd00027': (-5, 100)})

In [27]:
from modelseedpy import MSGapfill
gapfill = MSGapfill(model_base, default_gapfill_templates=[template_gramneg], test_conditions=tests, default_target='bio1')

In [28]:
gapfill_res = gapfill.run_gapfilling(media_glucose)

In [29]:
gapfill_res

{'reversed': {},
 'new': {'EX_cpd01981_e0': '>',
  'rxn05459_c0': '>',
  'rxn05481_c0': '<',
  'rxn00599_c0': '>',
  'rxn02185_c0': '<'},
 'media': <modelseedpy.core.msmedia.MSMedia at 0x7fd091075630>,
 'target': 'bio1',
 'minobjective': 0.01,
 'binary_check': False}

In [32]:
def _integrate_solution(template, model, gap_fill_solution):
    added_reactions = []
    for rxn_id, (lb, ub) in gap_fill_solution.items():
        template_reaction = template.reactions.get_by_id(rxn_id)
        model_reaction = template_reaction.to_reaction(model)
        model_reaction.lower_bound = lb
        model_reaction.upper_bound = ub
        _str = model_reaction.build_reaction_string(True)
        #print(f'{model.id} add {model_reaction.id}: {_str}')
        added_reactions.append(model_reaction)
    model.add_reactions(added_reactions)
    add_exchanges = MSBuilder.add_exchanges_to_model(model)

    return added_reactions, add_exchanges

model_gapfilled = model_base.copy()

from modelseedpy.core.msmodel import get_reaction_constraints_from_direction
from modelseedpy import MSBuilder
gap_sol = {}
for rxn_id, d in gapfill_res['new'].items():
    if rxn_id[:-1] in template_gramneg.reactions:
        gap_sol[rxn_id[:-1]] = get_reaction_constraints_from_direction(d)
print(gap_sol)
_integrate_solution(template_gramneg, model_gapfilled, gap_sol)

{'rxn05459_c': (0, 1000), 'rxn05481_c': (-1000, 0), 'rxn00599_c': (0, 1000), 'rxn02185_c': (-1000, 0)}


([<Reaction rxn05459_c0 at 0x7fcb9f6ed120>,
  <Reaction rxn05481_c0 at 0x7fcb9f6ed390>,
  <Reaction rxn00599_c0 at 0x7fcabde68100>,
  <Reaction rxn02185_c0 at 0x7fcabde68790>],
 [<Reaction EX_cpd01981_e0 at 0x7fcabde689d0>])

In [37]:
import math
def integrate_to_model_medium(media, model, prefix='EX_'):
    medium = {}
    for cpd, (lb, ub) in media.get_media_constraints().items():
        rxn_exchange = f'{prefix}{cpd}'
        if rxn_exchange in model.reactions:
            medium[rxn_exchange] = math.fabs(lb)
        else:
            print('not in model', cpd)
    return medium
model_gapfilled.medium = integrate_to_model_medium(media_glucose, model_gapfilled)
model_gapfilled.objective = 'bio1'
model_gapfilled.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00007_e0,EX_cpd00007_e0,8.362,0,0.00%
cpd00009_e0,EX_cpd00009_e0,0.3665,0,0.00%
cpd00013_e0,EX_cpd00013_e0,4.431,0,0.00%
cpd00027_e0,EX_cpd00027_e0,5,6,100.00%
cpd00030_e0,EX_cpd00030_e0,0.0008871,0,0.00%
cpd00034_e0,EX_cpd00034_e0,0.0008871,0,0.00%
cpd00048_e0,EX_cpd00048_e0,0.1153,0,0.00%
cpd00058_e0,EX_cpd00058_e0,0.0008871,0,0.00%
cpd00063_e0,EX_cpd00063_e0,0.0008871,0,0.00%
cpd00099_e0,EX_cpd00099_e0,0.0008871,0,0.00%
